<a href="https://colab.research.google.com/github/llindish/Complete-Python-3-Bootcamp/blob/master/WTD_model/GEE_code/Lochhills_S2_ANNUAL_mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

ee.Authenticate() # execute when running the first time

In [2]:
#Specify the Google Cloud Storage project you are going to be using
cloud_project = 'geographic-information-systems'
ee.Initialize(project=cloud_project)
geemap.ee_initialize(project=cloud_project)

In [6]:
#ANNUAL Sentinel-2 imagery with CS+ mask for Lochhills

###########################
#ADJUST FOLLOWING 4 variable
# Specify the asset (only for visualization at the last part here on GEEmap, can be skipped)
#asset (.shp file of the loggers stored on GEE as asset or coordnates)
asset_id = ee.Geometry.Point([-1.981705, 57.582320])

# Specify the year.
year = 2023

# Specify the name of your bucket (where would you like to export) and the specific location in the bucket
bucket = 'soilmoisture-jbdsag'
path = 'GEE_exports_LT/S2/Lochhills/Brit_grid_27700/annual_mosaic'

# Make sure the asset is a feature collection
geometry = ee.FeatureCollection(asset_id)

# !Specify the boundary box (the images area is based on these coordinates)
coords = [[-1.987558478776772,57.5800104255018],
[-1.975670928422768,57.5800104255018],
[-1.975670928422768,57.58499128012918],
[-1.987558478776772,57.58499128012918],
[-1.987558478776772,57.5800104255018]]

###########################

# Cloud Score+ image collection.
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

# Harmonized Sentinel-2 Level 2A collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Link S2 and CS+ results.
linkedCollection = s2.linkCollection(csPlus, ['cs', 'cs_cdf'])

# Masks pixels with low CS+ QA scores.
def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.60
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)

# NDVI and NDWI calculation function.
def addIndices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B8', 'B3']).rename('NDWI')
    return image.addBands(ndvi).addBands(ndwi)

# Create the polygon
aoi = ee.Geometry.Polygon(coords)

# Convert the year to ee.Date objects
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)

# Apply the cloud mask and NDVI calculation.
withIndices = linkedCollection.filterBounds(aoi).filterDate(start_date, end_date).map(maskLowQA).map(addIndices)

# Function to compute the annual mean
def annual_mean(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    annual_images = withIndices.filterDate(start_date, end_date)
    mean_image = annual_images.mean()
    min_image = annual_images.select(['NDVI', 'NDWI']).reduce(ee.Reducer.min())
    max_image = annual_images.select(['NDVI', 'NDWI']).reduce(ee.Reducer.max())
    range_image = max_image.subtract(min_image).rename(['NDVI_range', 'NDWI_range'])
    return mean_image.set('system:time_start', start_date.millis(), 'year', year)

# Create a list of years
years = ee.List.sequence(year, year)

# Map the function over the list of years to get a new ImageCollection
annual_means = ee.ImageCollection.fromImages(years.map(lambda y: annual_mean(y)))

# Export each image in the ImageCollection
def export_image(img):
    """Inner function to export a single image."""
    year = ee.Number(img.get('year')).format('%04d').getInfo()
    # Construct the file name prefix.
    #SPECIFY THE FILE NAME
    file_name_prefix = f"{path}/Lochhills_S2_annual_{year}"
    print(f"Exporting {file_name_prefix}")
    # Cast all bands to Float32.
    img = img.float()
    export_options = {
        'image': img,
        'bucket': bucket_name,
        'fileNamePrefix': file_name_prefix,
        'region': aoi,
        'scale': 10,
        'maxPixels': 1e10,
        'crs': 'EPSG:27700'
    }
    task = ee.batch.Export.image.toCloudStorage(**export_options)
    task.start()

# Initialize your ImageCollection
image_collection = annual_means
# Specify your bucket name
bucket_name = bucket
folder_path = path

# Call the function
annual_means_list = annual_means.toList(annual_means.size())
for i in range(annual_means_list.size().getInfo()):
    img = ee.Image(annual_means_list.get(i))
    export_image(img)

Exporting GEE_exports_LT/S2/Lochhills/Brit_grid_27700/annual_mosaic/Lochhills_S2_annual_2021


In [4]:
#Visualisatioz on GEEmap

map = geemap.Map()
url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
map.add_tile_layer(url, name="Google Satellite", attribution="Google")

# Centre the map view over the ROI
map.centerObject(geometry, 14)

# Get the image
image_mosaic = ee.Image(annual_means.filter(ee.Filter.eq('year', year)).first())

# Define visualization parameters
viz_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add the NDVI layer to the map
map.addLayer(image_mosaic.select('NDVI'), viz_params, 'NDVI')
map.addLayer(geometry, {'color': 'red'}, 'Loggers')

map

Map(center=[57.58232000000001, -1.981705], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…